For Portfolio Dashboard Statistic cards, Speedometer gauges and Debrief Room display

This notebook:
A concise pipeline that parses each trade’s `ID` to its underlying, fetches daily OHLC via yfinance, and computes a running **S\$** portfolio equity from **realised cashflows** (`Net Amount (S$) (Fuel)`), **ignoring `open` rows**; it outputs a CSV augmented with OHLC, date used, Yahoo symbol, and equity, with a **configurable starting capital** for portfolio analysis by LLM.

1. Loads your **Trades History.csv** from C:\Temp\Dashboard\data\
2. Add OHLC and Equity columns
3. Exports CSV outputs to C:\Temp\Dashboard\data\f1_dashboard_outputs

In [2]:
"""
Augments a trades history CSV with daily OHLC (from ID→underlying),
and computes PORTFOLIO_EQUITY using ONLY the 'Net Amount (S$) (Fuel)' column,
ignoring rows where Status == 'open' (case/whitespace insensitive).

Robust parsing:
- Status: .strip().lower() == 'open' → ignored in equity
- Net Amount (S$) (Fuel): handles parentheses negatives, commas, stray symbols

Equity (in S$):
  PORTFOLIO_EQUITY = STARTING_CAPITAL + cumulative sum of parsed net cashflows (closed rows only)
Sorted by Date then original row order for stability.

Also includes:
- Robust ID parsing (equities, index options, OCC-like forms)
- yfinance MultiIndex flattening → simple OHLC fields
- numeric coercion for OHLC
- Column renames you requested

Author-configurable:
- STARTING_CAPITAL
- output folder
- fill method for price date selection
"""

import re
import sys
import time
import typing as T
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

# =========================
# User configuration
# =========================
TRADES_PATH = r"C:\Temp\Dashboard\data\Trades History.csv"   # adjust if needed

# Custom output folder on Windows (use raw string to avoid escape issues)
OUT_DIR = Path(r"C:\Temp\Dashboard\data\f1_dashboard_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH = OUT_DIR / "Trades History_with_OHLC.csv"

# Price-date selection for OHLC:
# 'previous' -> last trading day <= trade date
# 'nearest'  -> nearest trading day to trade date (before/after)
FILL_METHOD = "previous"
LEFT_BUFFER_DAYS  = 10
RIGHT_BUFFER_DAYS = 5

# Portfolio equity settings (in S$ to match cashflow column)
STARTING_CAPITAL  = 10_000.0     # <-- change here if desired

# Column with net cashflow in S$
NET_CASH_COL = "Net Amount (S$) (Fuel)"

# =========================
# Index mapping
# =========================
INDEX_UNDERLYING_MAP = {
    "SPX": "^GSPC",
    "SPXW": "^GSPC",  # weeklies map to SPX
    # Extend if needed: "NDX": "^NDX", "RUT": "^RUT"
}

# =========================
# Helpers
# =========================
def derive_underlying_from_id(id_str: str) -> str:
    """Extract logical underlying from ID (plain tickers, equity/index options, OCC-like)."""
    if not id_str or not isinstance(id_str, str):
        return ""
    s = id_str.strip().upper()
    base = s.split()[0]
    # OCC-like no-space: NVDA251219C00130000
    if re.match(r"^[A-Z]+[0-9]{6}[CP][0-9A-Za-z]+$", base):
        m = re.match(r"^([A-Z]+)[0-9]{6}[CP].+$", base)
        if m:
            base = m.group(1)
    if base == "SPXW":
        base = "SPX"
    return base

def normalize_to_yahoo(symbol: str) -> str:
    """Normalize logical underlying to Yahoo-compatible ticker."""
    if not symbol:
        return ""
    s = symbol.strip().upper()
    if s in INDEX_UNDERLYING_MAP:
        return INDEX_UNDERLYING_MAP[s]
    if s in {"BTCUSD", "BTC-USD"}:
        return "BTC-USD"
    if s in {"ETHUSD", "ETH-USD"}:
        return "ETH-USD"
    if "." in s:  # BRK.B -> BRK-B
        left, right = s.split(".", 1)
        if len(right) <= 3:
            return f"{left}-{right}"
    return s

def _flatten_yf_df(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten yfinance MultiIndex columns to simple OHLC fields."""
    if df is None or df.empty:
        return pd.DataFrame(columns=["Open","High","Low","Close","Adj Close"])
    df = df.copy()
    df.index = pd.to_datetime(df.index)
    if isinstance(df.columns, pd.MultiIndex):
        if len(df.columns.levels[0]) == 1 and "Open" not in df.columns.levels[0]:
            df.columns = df.columns.droplevel(0)
        elif len(df.columns.levels[1]) == 1 and "Open" not in df.columns.levels[1]:
            df.columns = df.columns.droplevel(1)
        else:
            for lvl in [0, 1]:
                if set(["Open","High","Low","Close","Adj Close"]).issubset(
                    set(df.columns.get_level_values(lvl))
                ):
                    other = 1 - lvl
                    if len(df.columns.levels[other]) == 1:
                        df.columns = df.columns.droplevel(other)
                    break
    keep = [c for c in ["Open","High","Low","Close","Adj Close"] if c in df.columns]
    if not keep:
        cols_lower = {c.lower(): c for c in df.columns}
        mapped = {k: cols_lower.get(k.lower()) for k in ["Open","High","Low","Close","Adj Close"]}
        keep = [mapped[k] for k in ["Open","High","Low","Close","Adj Close"] if mapped.get(k)]
    df = df[keep].sort_index()
    return df

def _to_float(x):
    """Return float from scalar or 1-element Series; else NaN."""
    if isinstance(x, pd.Series):
        x = x.dropna()
        if not x.empty:
            return float(x.iloc[0])
        return np.nan
    try:
        return float(x)
    except (TypeError, ValueError):
        return np.nan

def _parse_net_amount_sgd(series: pd.Series) -> np.ndarray:
    """
    Parse 'Net Amount (S$) (Fuel)' robustly:
      - Trim spaces
      - Parentheses negatives: (1,234.56) -> -1234.56
      - Remove thousands separators, currency letters/symbols
    Returns float numpy array.
    """
    s = series.astype(str).str.strip()
    # (123.45) -> -123.45
    s = s.str.replace(r"^\((.*)\)$", r"-\1", regex=True)
    # Remove commas
    s = s.str.replace(",", "", regex=False)
    # Remove everything except digits, dot, minus
    s = s.str.replace(r"[^\d\.\-]+", "", regex=True)
    vals = pd.to_numeric(s, errors="coerce").fillna(0.0).values
    return vals

# =========================
# Data source: yfinance
# =========================
def fetch_ohlc_yf(symbols: T.List[str], start: datetime, end: datetime) -> dict:
    try:
        import yfinance as yf
    except ImportError:
        raise SystemExit("Please install yfinance: pip install yfinance --upgrade")
    out = {}
    for ysym in symbols:
        try:
            df = yf.download(
                ysym,
                start=start.strftime("%Y-%m-%d"),
                end=(end + timedelta(days=1)).strftime("%Y-%m-%d"),
                progress=False,
                auto_adjust=False,
                group_by="column",
                threads=True,
            )
            out[ysym] = _flatten_yf_df(df)
        except Exception as e:
            print(f"[WARN] Failed to fetch {ysym}: {e}", file=sys.stderr)
            out[ysym] = pd.DataFrame(columns=["Open","High","Low","Close","Adj Close"])
        time.sleep(0.2)
    return out

# =========================
# Locator for trade-day OHLC
# =========================
def pick_trade_day_price(df: pd.DataFrame, trade_date: pd.Timestamp, method: str):
    """Return numeric (O,H,L,C,AdjC, used_date) for the chosen trading day."""
    if df is None or df.empty:
        return (np.nan, np.nan, np.nan, np.nan, np.nan, pd.NaT)
    df = df.sort_index()
    if trade_date in df.index:
        row = df.loc[trade_date]
        return (_to_float(row.get("Open")),
                _to_float(row.get("High")),
                _to_float(row.get("Low")),
                _to_float(row.get("Close")),
                _to_float(row.get("Adj Close")),
                trade_date.normalize())
    if method == "previous":
        idx = df.index.searchsorted(trade_date, side="right") - 1
        if idx >= 0:
            used = df.index[idx]; row = df.iloc[idx]
            return (_to_float(row.get("Open")), _to_float(row.get("High")),
                    _to_float(row.get("Low")),  _to_float(row.get("Close")),
                    _to_float(row.get("Adj Close")), used.normalize())
        idx_after = df.index.searchsorted(trade_date, side="left")
        if idx_after < len(df):
            used = df.index[idx_after]; row = df.iloc[idx_after]
            return (_to_float(row.get("Open")), _to_float(row.get("High")),
                    _to_float(row.get("Low")),  _to_float(row.get("Close")),
                    _to_float(row.get("Adj Close")), used.normalize())
        return (np.nan, np.nan, np.nan, np.nan, np.nan, pd.NaT)
    elif method == "nearest":
        idx_right = df.index.searchsorted(trade_date, side="left")
        cands = []
        if idx_right < len(df): cands.append(df.index[idx_right])
        if idx_right > 0:       cands.append(df.index[idx_right-1])
        if not cands:
            return (np.nan, np.nan, np.nan, np.nan, np.nan, pd.NaT)
        used = min(cands, key=lambda d: abs(d - trade_date))
        row = df.loc[used]
        return (_to_float(row.get("Open")), _to_float(row.get("High")),
                _to_float(row.get("Low")),  _to_float(row.get("Close")),
                _to_float(row.get("Adj Close")), used.normalize())
    else:
        raise ValueError(f"Unknown FILL_METHOD: {method}")

# =========================
# Main
# =========================
def main():
    trades = pd.read_csv(TRADES_PATH)
    trades["Date"] = pd.to_datetime(trades["Date"], errors="coerce").dt.normalize()

    # Verify cashflow column exists
    if NET_CASH_COL not in trades.columns:
        raise ValueError(f"Required column not found: '{NET_CASH_COL}'")

    # === Underlying and Yahoo symbol ===
    trades["UNDERLYING_RAW"] = trades["ID"].astype(str).map(derive_underlying_from_id)
    trades["YF_SYMBOL"] = trades["UNDERLYING_RAW"].map(normalize_to_yahoo)

    # Log sample mappings
    print("\n=== ID → Underlying → Yahoo Symbol (sample) ===")
    for _, row in trades.head(10).iterrows():
        print(f"{row['ID']:30s} -> {row['UNDERLYING_RAW']:16s} -> {row['YF_SYMBOL']}")

    # === Fetch OHLC with buffer window ===
    min_dt = trades["Date"].min() - timedelta(days=LEFT_BUFFER_DAYS)
    max_dt = trades["Date"].max() + timedelta(days=RIGHT_BUFFER_DAYS)

    unique_syms = sorted(s for s in trades["YF_SYMBOL"].dropna().unique().tolist() if s)
    ohlc_map = fetch_ohlc_yf(unique_syms, start=min_dt, end=max_dt)

    # === Attach OHLC ===
    used_dates, o_list, h_list, l_list, c_list, ac_list = [], [], [], [], [], []
    for _, r in trades.iterrows():
        ysym = r["YF_SYMBOL"]; tdate = r["Date"]
        df = ohlc_map.get(ysym)
        O,H,L,C,AC,used_dt = pick_trade_day_price(df, pd.Timestamp(tdate), FILL_METHOD)
        used_dates.append(used_dt)
        o_list.append(O); h_list.append(H); l_list.append(L); c_list.append(C); ac_list.append(AC)

    trades["PX_DATE_USED"]  = pd.to_datetime(used_dates)
    trades["PX_OPEN"]       = o_list
    trades["PX_HIGH"]       = h_list
    trades["PX_LOW"]        = l_list
    trades["PX_CLOSE"]      = c_list
    trades["PX_ADJ_CLOSE"]  = ac_list

    # === Portfolio equity using 'Net Amount (S$) (Fuel)' ===
    cashflow_sgd_all = _parse_net_amount_sgd(trades[NET_CASH_COL])

    # Ignore 'open' rows (strip + lower)
    if "Status" in trades.columns:
        status_norm = trades["Status"].astype(str).str.strip().str.lower()
        closed_mask = status_norm != "open"
    else:
        closed_mask = pd.Series(True, index=trades.index)

    cashflow_used = np.where(closed_mask.values, cashflow_sgd_all, 0.0)

    # Order by Date then original order to compute cumulative equity
    trades["_ROW_"] = np.arange(len(trades))
    sort_idx = np.lexsort((trades["_ROW_"].values, trades["Date"].values.astype("datetime64[ns]")))
    equity_sorted = STARTING_CAPITAL + np.cumsum(cashflow_used[sort_idx])

    inv_idx = np.empty_like(sort_idx)
    inv_idx[sort_idx] = np.arange(len(sort_idx))
    trades["PORTFOLIO_EQUITY"] = equity_sorted[inv_idx].astype(float)
    trades.drop(columns=["_ROW_"], inplace=True)

    # === Diagnostics ===
    n_open_ignored = int((~closed_mask).sum())
    total_used = float(np.nansum(cashflow_used))
    print(f"\nDiagnostics:")
    print(f"  Closed rows used for equity: {int(closed_mask.sum())}")
    print(f"  Open rows ignored:           {n_open_ignored}")
    print(f"  Sum of used cashflows (S$):  {total_used:,.2f}")
    if np.allclose(cashflow_used, 0.0):
        # Help the user spot parsing issues quickly
        print("\nWARNING: All parsed cashflows are 0. Verify formatting of 'Net Amount (S$) (Fuel)'.")
        print("Sample raw values:")
        print(trades[NET_CASH_COL].head(10).to_list())

    print(f"Starting capital (S$):         {STARTING_CAPITAL:,.2f}")

    # === Rename columns per your request ===
    rename_map = {
        "UNDERLYING_RAW": "UNDERLYING_TICKER",
        "PX_DATE_USED":   "DATE_USED",
        "PX_OPEN":        "OPEN",
        "PX_HIGH":        "HIGH",
        "PX_LOW":         "LOW",
        "PX_CLOSE":       "CLOSE",
        "PX_ADJ_CLOSE":   "ADJ_CLOSE",
    }
    trades = trades.rename(columns=rename_map)

    trades.to_csv(OUTPUT_PATH, index=False)
    print(f"\nDone. Wrote: {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


C:\Users\boonm\AppData\Local\Temp\ipykernel_8816\228389248.py:238: UserWarning: Parsing dates in %d %m %Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  trades["Date"] = pd.to_datetime(trades["Date"], errors="coerce").dt.normalize()



=== ID → Underlying → Yahoo Symbol (sample) ===
NVDA 251219C00130000           -> NVDA             -> NVDA
DECK                           -> DECK             -> DECK
TSLA                           -> TSLA             -> TSLA
PLTR                           -> PLTR             -> PLTR
PLTR                           -> PLTR             -> PLTR
PLTR                           -> PLTR             -> PLTR
PLTR                           -> PLTR             -> PLTR
PLTR                           -> PLTR             -> PLTR
PLTR                           -> PLTR             -> PLTR
META                           -> META             -> META

Diagnostics:
  Closed rows used for equity: 100
  Open rows ignored:           21
  Sum of used cashflows (S$):  3,817.15
Starting capital (S$):         10,000.00

Done. Wrote: C:\Temp\Dashboard\data\f1_dashboard_outputs\Trades History_with_OHLC.csv


For Portfolio Dashboard F1 Grand Prix Circuit Chart and Championship Standings display

This notebook:
1. Loads your **Trades History.csv** from C:\Temp\Dashboard\data\
2. Computes overall performance metrics 
3. Computes points based on F1 points system and performance metric per Strategy
5. Exports CSV outputs to C:\Temp\Dashboard\data\f1_dashboard_outputs

**Instructions:** Place `Trades History.csv` in the same folder as this notebook. Then run cells top-to-bottom.

In [3]:
# ===================== F1 Dashboard: CSV Pipeline (Updated Points System) =====================
# Portfolio (Telemetry): Date-Sharpe from observed closed-trade dates (non-annualised, rf=0, ddof=1)
# Strategy (Podium):     Trade-Sharpe per round-trip (non-annualised, rf=0, ddof=1), CAGR annualised, ROI vs fixed start cap
# Calmar removed. New points:
#   Core (scaled): Sharpe 25, TE 18, CAGR 15
#   Bonuses: ROI +4 (highest), WinRate +4 (highest), MDD +4 (lowest |MDD|)
# -----------------------------------------------------------------------------------------

import pandas as pd, numpy as np
from pathlib import Path

# -------------------------- Config --------------------------
DAYFIRST                   = True
PORTFOLIO_STARTING_CAPITAL = 10_000.0
PER_STRATEGY_STARTING_CAP  = 10_000.0

# Sharpe RF = 0 (non-annualised)
RISK_FREE_RATE_MODE  = "annualized"   # "daily" or "annualized" (kept for completeness)
RISK_FREE_RATE_VALUE = 0.00
RF_DAILY = float(RISK_FREE_RATE_VALUE) if RISK_FREE_RATE_MODE.lower() == "daily" else float(RISK_FREE_RATE_VALUE)/252.0

# ------------------ Points (UPDATED) ------------------
# Core points (scaled to best performer; negatives clamp to 0)
W_SHARPE = 25
W_TE     = 18
W_CAGR   = 15

# Bonuses
BONUS_ROI_MAX         = 4   # highest ROI
BONUS_WINRATE_MAX     = 4   # highest WinRate
BONUS_MDD_LOWEST_ABS  = 4   # lowest |MDD|

# ------------------ Locate inputs -------------------
CWD = Path.cwd()
candidates = [
    CWD / "Trades History.csv",
    CWD.parent / "Trades History.csv",
    CWD / "data" / "Trades History.csv",
    CWD.parent / "data" / "Trades History.csv",
    Path("C:/Temp/Dashboard/data/Trades History.csv"),
]
TRADES_FILE = next((p for p in candidates if p.exists()), None)
if TRADES_FILE is None:
    raise FileNotFoundError("Could not find 'Trades History.csv'")

ROOT = TRADES_FILE.parent.parent if TRADES_FILE.parent.name.lower() == "data" else TRADES_FILE.parent
DATA_DIR = ROOT / "data"
OUTDIR   = DATA_DIR / "f1_dashboard_outputs"
OUTDIR.mkdir(parents=True, exist_ok=True)

# ------------------------------ Helpers ---------------------------------------
def parse_money_sgd(x):
    if pd.isna(x): return np.nan
    s = str(x).replace('S$','').replace('$','').replace(',','').strip()
    s = s.replace('(', '-').replace(')', '')
    try: return float(s)
    except ValueError: return np.nan

def ensure_columns(df, req):
    miss = [c for c in req if c not in df.columns]
    if miss: raise ValueError(f"Missing required columns: {miss}")

def daily_sharpe(returns, rf_daily=0.0):
    """Sharpe using non-annualised returns and sample std (ddof=1)."""
    r = pd.Series(returns).dropna()
    if r.empty or r.std(ddof=1) == 0: return np.nan
    excess = r - rf_daily
    return float(excess.mean() / r.std(ddof=1))

def max_drawdown_with_dates(equity_series):
    """Return (max_drawdown, peak_date, trough_date, recovery_date) on observed index."""
    s = pd.Series(equity_series).dropna().astype(float)
    if s.empty: return np.nan, None, None, None
    
    # Ensure unique index by taking the last value for each date
    s = s.groupby(s.index).last()
    
    roll_max = s.cummax()
    dd = (s / roll_max) - 1.0
    trough_date = dd.idxmin()
    if pd.isna(trough_date): return np.nan, None, None, None
    peak_date = s.loc[:trough_date].idxmax()
    rec_slice = s.loc[trough_date:]
    rec_mask = rec_slice >= s.loc[peak_date]
    recovery_date = rec_mask[rec_mask].index.min() if rec_mask.any() else None
    
    # Ensure we return a scalar value
    max_dd_value = dd.loc[trough_date]
    if hasattr(max_dd_value, '__len__') and len(max_dd_value) > 1:
        max_dd_value = max_dd_value.iloc[-1]  # Take the last value if still multiple
    
    return float(max_dd_value), peak_date, trough_date, recovery_date

def annualize_from_equity(equity_series):
    """CAGR from equity series using calendar years between first/last observed dates."""
    s = pd.Series(equity_series).dropna().astype(float)
    if len(s) < 2 or s.iloc[0] <= 0: return np.nan
    years = (s.index[-1] - s.index[0]).days / 365.25
    if years <= 0: return np.nan
    return float((s.iloc[-1] / s.iloc[0])**(1/years) - 1.0)

def clamp_nonneg(x): return np.maximum(np.asarray(x, dtype=float), 0.0)

def scaled_points(series, weight):
    """Scale non-negative metric to [0, weight] by dividing by max."""
    s = pd.Series(clamp_nonneg(series))
    m = s.max(skipna=True)
    if not np.isfinite(m) or m <= 0:
        return pd.Series(0.0, index=s.index)
    return weight * (s / m)

# ------------------------------ Load trades -----------------------------------
tr_raw = pd.read_csv(TRADES_FILE, encoding="utf-8")
rename_map = {
    "Net Amount (S$) (Fuel)": "Net Amount (S$)",
    "Strategy (Car)": "Strategy",
    "Risk tolerance (Driver)": "Driver",
    "Acct ID (Team)": "Team",  # Added this mapping for the team column
    "Team": "Team",
}
tr = tr_raw.rename(columns={k:v for k,v in rename_map.items() if k in tr_raw.columns})
ensure_columns(tr, ['Date','Type','ID','Net Amount (S$)','Strategy','Status'])
if 'Team' not in tr.columns: tr['Team'] = 'Default'

tr['Date'] = pd.to_datetime(tr['Date'], dayfirst=DAYFIRST, errors='coerce')
tr['NetAmount_SGD'] = tr['Net Amount (S$)'].apply(parse_money_sgd)

is_closed = tr['Status'].astype(str).str.strip().str.lower().eq('closed')
tr_closed = tr[is_closed].copy()
tr_open   = tr[~is_closed].copy()

# ---------------------- Round-trip trade-level ----------------------
def has_both_sides(g):
    sides = g['Type'].astype(str).str.strip().str.lower().unique()
    return ('buy' in sides) and ('sell' in sides)

rows = []
for (iid, strat), g in tr_closed.groupby(['ID','Strategy'], dropna=False):
    if has_both_sides(g):
        # Get the team for this trade (should be consistent within the trade)
        team = g['Team'].iloc[0] if not g['Team'].empty else 'Default'
        rows.append({
            'ID': iid,
            'Strategy': strat,
            'Team': team,  # Added team to trade level data
            'CloseDate': g['Date'].max(),
            'TradePnL_SGD': g['NetAmount_SGD'].sum(),
            'Legs': len(g),
        })
trade_level = pd.DataFrame(rows).sort_values(['CloseDate','Strategy','ID'])
trade_level.to_csv(OUTDIR / 'trade_level_pnl.csv', index=False)

if trade_level.empty:
    raise ValueError("No closed round-trip trades found.")

# ---------------------- Portfolio (Telemetry) ----------------------
# Equity = starting capital + cumulative closed-leg PnL (observed closed-trade dates only)
daily_closed_all = (
    tr_closed.groupby('Date', as_index=True)['NetAmount_SGD']
             .sum().sort_index()
)
equity_daily = (PORTFOLIO_STARTING_CAPITAL + daily_closed_all.cumsum()).rename('Equity')

print(f"\n[DEBUG] equity_daily has {len(equity_daily)} points "
      f"({equity_daily.index.min().date()} → {equity_daily.index.max().date()})")

# Portfolio returns on observed days; fix first return vs start cap; drop zero-PnL days
raw_returns = (equity_daily / equity_daily.shift(1) - 1.0)
first_date = equity_daily.index[0]
raw_returns.loc[first_date] = (equity_daily.iloc[0] - PORTFOLIO_STARTING_CAPITAL) / PORTFOLIO_STARTING_CAPITAL
nonzero_mask = daily_closed_all != 0
daily_ret_port = raw_returns.where(nonzero_mask, np.nan).dropna()

port_sharpe = daily_sharpe(daily_ret_port, rf_daily=0.0)

# Month-end equity for charts only
equity_monthly = equity_daily.resample('ME').last().dropna()
overall_df = equity_monthly.reset_index().rename(columns={'Date':'Month'})
overall_df.to_csv(OUTDIR / 'overall_equity_curve.csv', index=False)

# Portfolio headline metrics (UNCHANGED - no team info here as requested)
port_cagr = annualize_from_equity(equity_daily)
port_mdd, _, _, _ = max_drawdown_with_dates(equity_daily)

gp = trade_level['TradePnL_SGD'].clip(lower=0).sum()
gl = -trade_level['TradePnL_SGD'].clip(upper=0).sum()
port_pf = float(gp / gl) if gl > 0 else np.nan
port_wr = float((trade_level['TradePnL_SGD'] > 0).mean()) if len(trade_level) else np.nan

end_eq = float(equity_daily.iloc[-1])
roi = (end_eq - PORTFOLIO_STARTING_CAPITAL) / PORTFOLIO_STARTING_CAPITAL

portfolio_metrics = pd.DataFrame([{
    'StartDate': equity_daily.index[0].date(),
    'EndDate':   equity_daily.index[-1].date(),
    'StartEquity': PORTFOLIO_STARTING_CAPITAL,
    'EndEquity':   end_eq,
    'CAGR': port_cagr,
    'Sharpe': port_sharpe,          # Date-Sharpe (daily, ddof=1, rf=0)
    'MaxDrawdown': port_mdd,        # scalar only
    'WinRate': port_wr,
    'ProfitFactor': port_pf,
    'ROI': roi,                     # total (not annualised)
    'OpenTrades': int(tr_open['ID'].nunique()),
}])
portfolio_metrics.to_csv(OUTDIR / 'portfolio_metrics.csv', index=False)
print("[DEBUG] Wrote portfolio_metrics.csv")

# ---------------------- Strategy (Podium) ----------------------
strategy_rows = []
print("\n[DEBUG] --- Strategy breakdown (Sharpe/CAGR/ROI/TE, anchored to start cap) ---")

# Create a mapping of strategy to team for later use
strategy_team_map = trade_level.groupby('Strategy')['Team'].first().to_dict()

for strat, tl_s in trade_level.groupby('Strategy', dropna=False):
    tl_s = tl_s.sort_values('CloseDate').copy()

    total_pnl = tl_s['TradePnL_SGD'].sum() if not tl_s.empty else 0.0
    roi_s = total_pnl / PER_STRATEGY_STARTING_CAP

    if not tl_s.empty:
        close_dates = pd.to_datetime(tl_s['CloseDate'].values)
        cum_pnl     = tl_s['TradePnL_SGD'].cumsum().values
        eq_cum      = PER_STRATEGY_STARTING_CAP + cum_pnl

        first_dt    = pd.to_datetime(tl_s['CloseDate'].min())
        last_dt     = pd.to_datetime(tl_s['CloseDate'].max())

        # equity incl. start-cap anchor (for MDD baseline)
        # Handle potential duplicate dates by creating daily equity series
        equity_data = []
        equity_dates = []
        
        # Add starting capital
        start_anchor_date = first_dt - pd.Timedelta(days=1)
        equity_data.append(PER_STRATEGY_STARTING_CAP)
        equity_dates.append(start_anchor_date)
        
        # Add cumulative equity for each trade close date
        for i, date in enumerate(close_dates):
            equity_data.append(eq_cum[i])
            equity_dates.append(date)
        
        # Create series and handle duplicates by taking the last value for each date
        eq_with_start = pd.Series(data=equity_data, index=equity_dates)
        eq_with_start = eq_with_start.groupby(eq_with_start.index).last()

        # Trade-Sharpe (include first jump from start cap)
        first_ret  = (eq_cum[0] - PER_STRATEGY_STARTING_CAP) / PER_STRATEGY_STARTING_CAP
        subsequent = pd.Series(eq_cum).pct_change().iloc[1:].replace([np.inf, -np.inf], np.nan)
        ret_trade  = pd.concat([pd.Series([first_ret]), subsequent], ignore_index=True).dropna()
        sharpe_s   = daily_sharpe(ret_trade, rf_daily=0.0) if len(ret_trade) >= 2 else np.nan

        # CAGR (annualised) between first & last CLOSE (365.25)
        days        = (last_dt - first_dt).days
        years_36525 = days / 365.25 if days > 0 else np.nan
        cagr_s      = ( (eq_cum[-1] / PER_STRATEGY_STARTING_CAP) ** (1/years_36525) - 1
                       ) if pd.notna(years_36525) and years_36525 > 0 else np.nan

        # MDD (for MDD bonus only)
        mdd_s, _, _, _ = max_drawdown_with_dates(eq_with_start)
    else:
        first_dt = last_dt = None
        sharpe_s = cagr_s = mdd_s = np.nan

    print(f"[{strat}] Trades={len(tl_s)} | ROI={roi_s:.6f} | "
          f"CAGR={cagr_s if pd.notna(cagr_s) else np.nan:.6f} | "
          f"Sharpe(Trade)={sharpe_s if pd.notna(sharpe_s) else np.nan:.6f} | "
          f"MDD={mdd_s if pd.notna(mdd_s) else np.nan:.6f}")

    strategy_rows.append({
        'Strategy': strat,
        'Team': strategy_team_map.get(strat, 'Default'),  # Added team information
        'CAGR': cagr_s,         # annualised, relative to PER_STRATEGY_STARTING_CAP
        'Sharpe': sharpe_s,     # Trade-Sharpe (non-annualised, ddof=1, rf=0)
        'MDD': mdd_s,           # for bonus only
        'ROI': roi_s,           # for ROI bonus
    })

strategy_core = pd.DataFrame(strategy_rows)

# Trade-level stats per Strategy (for TE/WinRate)
trade_stats_strat = (
    trade_level.groupby('Strategy')
               .agg({
                   'TradePnL_SGD': ['sum',
                                    lambda s: (s > 0).sum(),
                                    lambda s: (s <= 0).sum(),
                                    'count',
                                    lambda s: s[s > 0].mean(),
                                    lambda s: -s[s <= 0].mean()]
               })
)
trade_stats_strat.columns = ['TotalPnL','Wins','Losses','Trades','AvgWin','AvgLoss']
trade_stats_strat = trade_stats_strat.fillna(0.0)
trade_stats_strat['WinRate']    = trade_stats_strat['Wins'] / trade_stats_strat['Trades']
trade_stats_strat['Expectancy'] = (trade_stats_strat['WinRate'] * trade_stats_strat['AvgWin']) - ((1 - trade_stats_strat['WinRate']) * trade_stats_strat['AvgLoss'])
trade_stats_strat['TE']         = trade_stats_strat['Expectancy'] * trade_stats_strat['Trades']
trade_stats_strat = trade_stats_strat.reset_index()

# Merge & score (NEW points)
strategy_points = strategy_core.merge(trade_stats_strat, on='Strategy', how='left')

# Core points (scaled to best performer; negatives -> 0)
def scaled(series, weight):
    s = pd.Series(series).copy()
    s = s.clip(lower=0)   # clamp negatives -> 0
    mx = s.max(skipna=True)
    return weight * (s / mx) if (pd.notna(mx) and mx > 0) else s*0

strategy_points['Sharpe_points'] = scaled(strategy_points['Sharpe'], W_SHARPE).fillna(0.0)
strategy_points['TE_points']     = scaled(strategy_points['TE'],     W_TE).fillna(0.0)
strategy_points['CAGR_points']   = scaled(strategy_points['CAGR'],   W_CAGR).fillna(0.0)

# Bonuses: ROI (highest), WinRate (highest), MDD (lowest |MDD|)
strategy_points['ROI_Bonus']     = (strategy_points['ROI'] == strategy_points['ROI'].max()).astype(float) * BONUS_ROI_MAX
strategy_points['WinRate_Bonus'] = (strategy_points['WinRate'] == strategy_points['WinRate'].max()).astype(float) * BONUS_WINRATE_MAX
min_abs_mdd = strategy_points['MDD'].abs().min() if strategy_points['MDD'].notna().any() else np.nan
strategy_points['MDD_Bonus']     = (strategy_points['MDD'].abs() == min_abs_mdd).astype(float) * BONUS_MDD_LOWEST_ABS
strategy_points[['ROI_Bonus','WinRate_Bonus','MDD_Bonus']] = strategy_points[['ROI_Bonus','WinRate_Bonus','MDD_Bonus']].fillna(0.0)

# Total points
point_cols = ['Sharpe_points','TE_points','CAGR_points','ROI_Bonus','WinRate_Bonus','MDD_Bonus']
strategy_points['TotalPoints'] = strategy_points[point_cols].sum(axis=1, skipna=True)
strategy_points = strategy_points.sort_values('TotalPoints', ascending=False).reset_index(drop=True)
strategy_points.insert(0, 'POS', range(1, len(strategy_points)+1))

# Reorder columns to put Team right after Strategy
cols = strategy_points.columns.tolist()
# Find the index of 'Strategy' and 'Team'
strategy_idx = cols.index('Strategy')
team_idx = cols.index('Team')

# Reorder to put Team right after Strategy
if team_idx != strategy_idx + 1:
    cols.pop(team_idx)  # Remove Team from its current position
    cols.insert(strategy_idx + 1, 'Team')  # Insert Team right after Strategy
    strategy_points = strategy_points[cols]

strategy_points.to_csv(OUTDIR / 'strategy_points.csv', index=False)

# ------------------------------ Summary --------------------------------------
print("\nSaved outputs to:", OUTDIR.resolve())
for f in [
    'trade_level_pnl.csv',
    'overall_equity_curve.csv',
    'portfolio_metrics.csv',
    'strategy_points.csv',
]:
    print(" -", f, "✓" if (OUTDIR / f).exists() else "✗")


[DEBUG] equity_daily has 40 points (2024-12-19 → 2025-09-23)
[DEBUG] Wrote portfolio_metrics.csv

[DEBUG] --- Strategy breakdown (Sharpe/CAGR/ROI/TE, anchored to start cap) ---
[Assignment] Trades=1 | ROI=0.005496 | CAGR=nan | Sharpe(Trade)=nan | MDD=0.000000
[Bear Call Spread] Trades=8 | ROI=0.027060 | CAGR=0.082515 | Sharpe(Trade)=3.905672 | MDD=0.000000
[Bull Put Spread] Trades=15 | ROI=0.003694 | CAGR=0.006313 | Sharpe(Trade)=0.021952 | MDD=-0.046222
[Iron Condor] Trades=2 | ROI=0.052028 | CAGR=1.798792 | Sharpe(Trade)=0.805414 | MDD=0.000000
[Long Call] Trades=1 | ROI=0.196509 | CAGR=nan | Sharpe(Trade)=nan | MDD=0.000000
[Short Call] Trades=5 | ROI=0.015830 | CAGR=0.046962 | Sharpe(Trade)=2.100745 | MDD=0.000000
[Short Put] Trades=10 | ROI=0.074236 | CAGR=0.145941 | Sharpe(Trade)=0.701309 | MDD=-0.003251
[Short Strangle] Trades=3 | ROI=0.006862 | CAGR=0.027828 | Sharpe(Trade)=0.153307 | MDD=-0.014833

Saved outputs to: C:\Temp\Dashboard\data\f1_dashboard_outputs
 - trade_level_p